In [0]:
SILVER_BASE = "abfss://silver@stretaildata123.dfs.core.windows.net"
GOLD_BASE   = "abfss://gold@stretaildata123.dfs.core.windows.net"


In [0]:
orders_df        = spark.read.format("delta").load(f"{SILVER_BASE}/orders")
order_items_df  = spark.read.format("delta").load(f"{SILVER_BASE}/order_items")
products_df     = spark.read.format("delta").load(f"{SILVER_BASE}/products")
aisles_df       = spark.read.format("delta").load(f"{SILVER_BASE}/aisles")
departments_df  = spark.read.format("delta").load(f"{SILVER_BASE}/departments")

###Dimension Tables

####dim_customer

In [0]:
dim_customer = (
    orders_df
    .select("user_id")
    .distinct()
)

dim_customer.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/dim_customer")


####dim_department

In [0]:
dim_department = departments_df

dim_department.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/dim_department")


####dim_aisle

In [0]:
dim_aisle = aisles_df

dim_aisle.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/dim_aisle")


####dim_product (conformed dimension)

In [0]:
dim_product = (
    products_df
    .join(aisles_df, "aisle_id", "left")
    .join(departments_df, "department_id", "left")
)

dim_product.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/dim_product")

####dim_date (derived)

In [0]:
from pyspark.sql.functions import col

dim_date = (
    orders_df
    .select(
        col("order_dow").alias("day_of_week"),
        col("order_hour_of_day").alias("hour_of_day")
    )
    .distinct()
)

dim_date.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/dim_date")


###Fact Tables


####fact_orders

In [0]:
from pyspark.sql.functions import countDistinct

fact_orders = (
    orders_df
    .groupBy("user_id")
    .agg(
        countDistinct("order_id").alias("total_orders")
    )
)

fact_orders.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/fact_orders")


####fact_order_products

In [0]:
from pyspark.sql.functions import sum as _sum, count

fact_order_products = (
    order_items_df
    .join(orders_df.select("order_id", "user_id"), "order_id", "inner")
    .join(products_df.select("product_id", "aisle_id", "department_id"), "product_id", "inner")
    .groupBy("product_id", "aisle_id", "department_id")
    .agg(
        count("*").alias("times_ordered"),
        _sum("reordered").alias("times_reordered")
    )
)

fact_order_products.write.format("delta").mode("overwrite").save(f"{GOLD_BASE}/fact_order_products")
# The error is due to attempting to group by columns not present in order_items_df before the join.
# The join with products_df brings in aisle_id and department_id, so the code is correct after the join.

####Register Gold Tables in Metastore

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_dim_customer
USING DELTA
LOCATION '{GOLD_BASE}/dim_customer'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_dim_product
USING DELTA
LOCATION '{GOLD_BASE}/dim_product'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_dim_aisle
USING DELTA
LOCATION '{GOLD_BASE}/dim_aisle'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_dim_department
USING DELTA
LOCATION '{GOLD_BASE}/dim_department'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_fact_orders
USING DELTA
LOCATION '{GOLD_BASE}/fact_orders'
""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS gold_fact_order_products
USING DELTA
LOCATION '{GOLD_BASE}/fact_order_products'
""")
